## Import Dependencies

In [1]:
import os
download_folder = os.path.expanduser("~")+"/Downloads/"

import calendar
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
from pandas import ExcelFile
from pandas import ExcelWriter
import xlsxwriter

## Read in the full export

In [2]:
export = pd.read_csv(f'{download_folder}orders_export.csv')

In [3]:
len(export)

53

## Convert the 'Created at' to a timestamp. Title 'Date Ordered'

In [4]:
timestamps = pd.to_datetime(export['Created at'])
export['Date Ordered'] = timestamps.apply(lambda x: datetime(x.year, x.month, x.day))

## Define Today

In [5]:
now = datetime.now()

today = datetime(now.year, now.month, now.day)
    
    #### Hardcoding today for dev
    #today = datetime(now.year, now.month, 30, 0, 0)

today_string = today.strftime("%Y-%m-%d")
today_string

'2020-04-02'

In [6]:
# is today Monday? 
today.weekday() == 0

False

## Filter based on the day of the week

In [7]:
# if today is Monday, we need saturday and sunday purchases
if today.weekday() == 0:
    sunday = export[export['Date Ordered'] == today - timedelta(days=1)]
    saturday = export[export['Date Ordered'] == today - timedelta(days=2)]
    orders = pd.concat([sunday, saturday], ignore_index=True)
else:
    orders = export[export['Date Ordered'] == today - timedelta(days=1)]

In [8]:
len(orders)

15

## Fill in missing values from row above (to make up for prob with shopify export)

In [9]:
# if cell is blank, fill with value from above it
orders = orders.fillna(method='ffill')

## Filter out pre-orders

In [10]:
# filter out the pre-orders
orders = orders[~orders['Lineitem name'].str.contains('pre-order') ]

## Create To Make list for cutters

In [11]:
# Create counts table for cutters
to_make = orders['Lineitem name'].value_counts()
to_make = to_make.reset_index().rename(columns={'index': 'Lineitem name', 'Lineitem name': 'Count'})

## Create list for instore Pick Ups

In [12]:
# Create list for in store pickup
searchfor = ['in-store', 'additional item']
pick_ups = orders[orders['Shipping Method'].str.contains('in-store')][['Date Ordered','Billing Name','Lineitem name','Shipping Method']]
pick_ups

,Date Ordered,Billing Name,Lineitem name,Shipping Method
39,2020-04-01,Jenny Ortiz,Steak Home Pack,in-store pickup
40,2020-04-01,Jenny Ortiz,Steak Home Pack,in-store pickup
41,2020-04-01,Michael Jarnagan,Pork & Chicken Pack,in-store pickup
42,2020-04-01,Michael Jarnagan,Large Home Pack,in-store pickup
43,2020-04-01,Alexander Perez,Large Home Pack,in-store pickup
44,2020-04-01,Alicia Corbett,Small Home Pack,in-store pickup
46,2020-04-01,Daniel Dallen,Steak Home Pack,in-store pickup
47,2020-04-01,David Amack,Pork & Chicken Pack,in-store pickup
49,2020-04-01,James Myatt,Small Home Pack,in-store pickup
50,2020-04-01,Paul Martinez,Steak Home Pack,in-store pickup


## Create List to go out for Delivery

In [13]:
# Create list for deliveries (sort by zip)
deliveries = orders[orders['Shipping Method'].str.contains('Delivery')][['Date Ordered',
                                                            'Lineitem name',
                                                            'Shipping Method',
                                                            'Billing Name', 
                                                            'Shipping Street',
                                                            'Shipping City', 
                                                            'Shipping Zip',
                                                            'Shipping Phone', 
                                                            'Notes',]].sort_values(by=['Shipping Zip'])
deliveries

,Date Ordered,Lineitem name,Shipping Method,Billing Name,Shipping Street,Shipping City,Shipping Zip,Shipping Phone,Notes
52,2020-04-01,Large Home Pack,South Central Local Delivery,Katherine Kuster,2412 Little John,Austin,'78704,'+1 956-220-9447,NaN
38,2020-04-01,Pork & Chicken Pack,Local Delivery,Marcus Mernitz,1000 E 43rd St,Austin,'78751,NaN,NaN
45,2020-04-01,Large Home Pack,North Austin Local Delivery,Jamie Lightner,"1500 E Parmer Lane, 723",Austin,'78753,'+1 956-220-9447,NaN
51,2020-04-01,Pork & Chicken Pack,North Austin Local Delivery,Matt Kalinowski,11400 Charolais Cove,Austin,'78758,'+1 956-220-9447,NaN


## Create list to be shipped UPS

In [14]:
# Create list for items to be shipped
to_ship = orders[orders['Shipping Method'].str.contains('UPS')][['Date Ordered',
                                                            'Lineitem name',
                                                            'Shipping Method',
                                                            'Billing Name',
                                                            'Shipping Name',
                                                            'Shipping Street',
                                                            'Shipping City', 
                                                            'Shipping Zip',
                                                            'Shipping Phone', 
                                                            'Notes',]]
to_ship

,Date Ordered,Lineitem name,Shipping Method,Billing Name,Shipping Name,Shipping Street,Shipping City,Shipping Zip,Shipping Phone,Notes


## Print all reports to Excel with tabs

In [15]:
# creating new xlsx with multiple sheets

with pd.ExcelWriter(f'{download_folder}Online Order Reports-{today_string}.xlsx') as writer:
    to_make.to_excel(writer, sheet_name='To Make', index=False)
    pick_ups.to_excel(writer, sheet_name='Pick Ups', index=False)
    deliveries.to_excel(writer,sheet_name='Deliveries', index=False)
    to_ship.to_excel(writer, sheet_name='To Ship', index=False)
